In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import math
import random

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images / 255
test_images = test_images / 255



new_train_images = np.zeros((50000,280,280))
new_train_labels = np.zeros((50000,14))

for i in range(len(new_train_images)):

    randx = random.randrange(0,280-28,1)
    randy = random.randrange(0,280-28,1)

    new_train_images[i,randx:randx+28,randy:randy+28] = train_images[i]
    new_train_labels[i,0] = randx
    new_train_labels[i,1] = randy 
    new_train_labels[i,2] = randx + 28
    new_train_labels[i,3] = randy + 28
    new_train_labels[i,4+train_labels[i]] = 1 



trainimagestensor = tf.convert_to_tensor(new_train_images, dtype=tf.float16)
trainlabelstensor = tf.convert_to_tensor(new_train_labels, dtype=tf.float16)




: 

In [4]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred):        
        
        truebox = y_true[0:4]
        truelabel = y_true[4:]

        predbox = y_pred[0:4]
        predlabel = y_pred[4:]
        
        log_y_pred = tf.math.log(predlabel)
        elements = -tf.math.multiply_no_nan(x=log_y_pred, y=truelabel)
        ce = tf.reduce_mean(tf.reduce_sum(elements,axis=1))
        bbe = (truebox[0]-predbox[0])**2 + (truebox[1]-predbox[1])**2 + (truebox[2]-predbox[2])**2 + (truebox[3]-predbox[3])**2

        return ce + bbe


model = tf.keras.models.Sequential()


model.add(tf.keras.layers.Conv2D(filters=2,kernel_size=(3,3),strides=1,padding="same",activation="relu", input_shape=(280,280,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation="relu"))
model.add(tf.keras.layers.Dense(14))
model.summary()


model.compile(
  optimizer=tf.keras.optimizers.Adam(0.001),
  loss=CustomLoss(),
  metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 280, 280, 2)       20        
                                                                 
 flatten (Flatten)           (None, 156800)            0         
                                                                 
 dense (Dense)               (None, 64)                10035264  
                                                                 
 dense_1 (Dense)             (None, 14)                910       
                                                                 
Total params: 10036194 (38.29 MB)
Trainable params: 10036194 (38.29 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model.fit(trainimagestensor, trainlabelstensor)

2023-11-06 15:57:12.975899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-11-06 15:57:13.325093: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-06 15:57:13.389038: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cd3f0b5ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-06 15:57:13.389092: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2023-11-06 15:57:13.405724: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-06 15:57:13.557031: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

313/313 [==============================] - 4s 9ms/step - loss: nan - accuracy: 0.4733
